In [1]:
import pandas as pd
import numpy as np
import re
from IPython.display import display, Markdown, HTML

In [210]:
infile = "../pyCIMS_model_description.xlsm"

NODE_COL = "Node"
MODEL_SHEET = "Model"
# TODO extract other string constants used below and place them in some constants here

mxl = pd.read_excel(infile, sheet_name=None, header=1)
mdf0 = mxl[MODEL_SHEET].replace({pd.np.nan: None})

display(Markdown("""
# pyCIMS model reader

The code here reads [{}]({}) and extracts the '{}' sheet into a Pandas DataFrame.
The dataframe is then split and and the parts are organized in dictionaries, see below.

Later the individual node and tech frames are analyzed for service connectivity information, 
enabling the construction of a graph.

When building the graph, it would probably be best to carry the original dataframe splits into the resulting data structure, to stay as close as possible to the original input model format.
""".format(infile, infile, MODEL_SHEET)))


# pyCIMS model reader

The code here reads [../pyCIMS_model_description.xlsm](../pyCIMS_model_description.xlsm) and extracts the 'Model' sheet into a Pandas DataFrame.
The dataframe is then split and and the parts are organized in dictionaries, see below.

Later the individual node and tech frames are analyzed for service connectivity information, 
enabling the construction of a graph.

When building the graph, it would probably be best to carry the original dataframe splits into the resulting data structure, to stay as close as possible to the original input model format.


In [199]:
# DataFrame helpers
def display_df(df):
    """Display None as blanks"""
    df = pd.DataFrame(df)
    if not df.empty:
        display(pd.DataFrame(df).replace({None: ''}))
def empty_rows(df, exclude_column="Node"):
    """Return bool array to flag all rows that have only None or False values, ignoring exclude_column"""
    return df.loc[:, df.columns != exclude_column].T.apply(any)

re_year = re.compile(r'^[0-9]{4}$')
def is_year(cn):
    """Check if input int or str is 4 digits [0-9] between begin ^ and end $ of string"""
    # unit test: assert is_year, 1900
    return bool(re_year.match(str(cn)))
def find_first(items, pred=bool, default=None):
    """Find first item for that pred is True"""
    return next(filter(pred, items), default)
def find_first_index(items, pred=bool):
    """Find index of first item for that pred is True"""
    return find_first(enumerate(items), lambda kcn: pred(kcn[1]))[0]
def get_node_cols(mdf, first_data_col_name="Node"):
    """Returns list of column names after 'Node' and a list of years that follow """
    node_col_idx = find_first_index(mdf.columns,
                                    lambda cn: first_data_col_name.lower() in cn.lower())
    relevant_columns = mdf.columns[node_col_idx:]
    year_or_not = list(map(is_year, relevant_columns))
    first_year_idx = find_first_index(year_or_not)
    last_year_idx = find_first_index(year_or_not[first_year_idx:],
                                     lambda b: not b) + first_year_idx
    # list(...)[a:][:b] extracts b elements starting at a
    year_cols = mdf.columns[node_col_idx:][first_year_idx:last_year_idx]
    return mdf.columns[node_col_idx:][:first_year_idx], year_cols

In [200]:
node_cols, year_cols = get_node_cols(mdf0)
all_cols = np.concatenate((node_cols, year_cols))
mdf = mdf0.loc[1:,all_cols] # drop irrelevant columns and skip first, empty row

In [201]:
node_rows = mdf.Node[~mdf.Node.isnull()] # does not work if node names have been filled in (i.e. don't run after next cell)
node_rows.index.name = "Row Number"
last_row = mdf.index[-1]
node_start_ends = zip(node_rows.index,
                      node_rows.index[1:].tolist() + [last_row])

In [202]:
node_dfs = {}
non_node_cols = mdf.columns != NODE_COL
for s, e in node_start_ends:
    node_name = mdf.Node[s]
    node_df = mdf.loc[s+1:e-1]
    node_df = node_df.loc[empty_rows(node_df), non_node_cols]
    # mdf.loc[s+1:e-1, "Node"] = node_name
    node_dfs[node_name] = node_df
# len(node_dfs)

In [203]:
## intermediate output for dev purposes
# for nn, ndf in node_dfs.items():
#     display(Markdown("Node: **{}**".format(nn)))
#     display_df(ndf)

In [204]:
tech_dfs = {}
for nn, ndf in node_dfs.items():
    if any(ndf.Parameter == "Technology"):
        tdfs = {}
        first_row, last_row = ndf.index[0], ndf.index[-1]
        tech_rows = ndf.loc[ndf.Parameter == "Technology"].index
        for trs, tre in zip(tech_rows, tech_rows[1:].tolist()+[last_row]):
            tech_df = ndf.loc[trs:tre-1]
            tech_name = tech_df.iloc[0].Value
            tdfs[tech_name] = tech_df
        tech_dfs[nn] = tdfs
        node_dfs[nn] = ndf.loc[:tech_rows[0]-1]

# Display Nodes and Technologies
`node_dfs` contains a dictionary giving the DataFrames that hold the relevant rows for each node, without Technology info  
`tech_dfs` contains a dictionary for each node naming the technologies and holding the relevant rows in a DF

In [206]:
for nn, ndf in node_dfs.items():
    display(Markdown("Node: **{}**".format(nn)))
    display_df(ndf)
    if nn in tech_dfs:
        for tech_name, tdf in tech_dfs[nn].items():
            display(Markdown("Node / Technology: **{} / {}**".format(nn, tech_name)))
            display_df(tdf)

Node: **Simulation**

,Parameter,Branch,Unit,Value,2000,2005,2010,2015,2020,2025,2030,2035,2040,2045,2050
2,Simulation Time,,Years,,2000.0,2005.0,2010.0,2015.0,2020.0,2025.0,2030.0,,,,


Node: **Canada**

,Parameter,Branch,Unit,Value,2000,2005,2010,2015,2020,2025,2030,2035,2040,2045,2050
5,Service supply,Canada,,,,,,,,,,,,,
6,Competition type,,,Fixed Ratio,,,,,,,,,,,
7,Service demand,Canada.Alberta,,,1,1,1,1,1,1,1,1,1,1,1


Node: **Alberta**

...

Node / Technology: **Conventional / Solar PV**

,Parameter,Branch,Unit,Value,2000,2005,2010,2015,2020,2025,2030,2035,2040,2045,2050
428,Technology,,,Solar PV,,,,,,,,,,,
429,Available,,Date,Solar PV,2005,,,,,,,,,,
430,Unavailable,,Date,Solar PV,2100,,,,,,,,,,
431,Lifetime,,Years,Solar PV,30,,,,,,,,,,
432,Financial discount rate,,%,Solar PV,0.125,,,,,,,,,,
433,Capital Cost,,2015$/GJ,Solar PV,4886,4641.7,4409.61,4189.13,3979.68,3780.69,3591.66,3412.08,3241.47,3079.4,2925.43
434,Operating Cost,,2015$/GJ,Solar PV,,,,,,,,,,,
435,Intangible Cost,,2015$/GJ,Solar PV,,,,,,,,,,,
436,Service Cost,Canada.Alberta.Electricity,2015$/GJ,Solar PV,11.572,,,,,,,,,,
437,Competition exclude,Canada.Alberta.Residential.Buildingss(Apartment),,Solar PV,,,,,,,,,,,


Node / Technology: **Conventional / Wind**

,Parameter,Branch,Unit,Value,2000,2005,2010,2015,2020,2025,2030,2035,2040,2045,2050
442,Technology,,,Wind,,,,,,,,,,,
443,Available,,Date,Wind,2000,,,,,,,,,,
444,Unavailable,,Date,Wind,2100,,,,,,,,,,
445,Lifetime,,Years,Wind,30,,,,,,,,,,
446,Financial discount rate,,%,Wind,0.125,,,,,,,,,,
447,Capital Cost,,2015$/GJ,Wind,15521,15521,15521,15521,15521,15521,15521,15521,15521,15521,15521
448,Operating Cost,,2015$/GJ,Wind,,,,,,,,,,,
449,Intangible Cost,,2015$/GJ,Wind,,,,,,,,,,,
450,Service Cost,Canada.Alberta.Electricity,2015$/GJ,Wind,8.8,,,,,,,,,,
451,Market share total_Max,,%,Wind,,,,,,,,,,,


Node: **Renewables**

# Display Service node paths and demand connections 

In [207]:
## simple node connectivity output
# for nn, ndf in node_dfs.items():
#     supply_path = ndf.loc[ndf.Parameter.str.lower() == "service supply", "Branch"]
#     if not supply_path.empty:
#         display(Markdown("## Node: **{}**".format(nn)))
#         display(Markdown("Path: **{}**".format(supply_path.values[0])))
#         if any(ndf.Parameter.str.lower() == "service demand"):
#             display(Markdown("### Service demand connections"))
#             display_df(ndf.loc[ndf.Parameter.str.lower() == "service demand", ["Branch","Value"]])
#         if nn in tech_dfs:
#             for tech_name, tdf in tech_dfs[nn].items():
#                 display(Markdown("### Technology service demand connections"))
#                 display(Markdown("Node / Technology: **{} / {}**".format(nn, tech_name)))
#                 display_df(tdf.loc[tdf.Parameter.str.lower() == "service demand", ["Branch","Value"]])
#                 # display_df(tdf)

In [208]:
def process_connection(con_path, con_name, what=""):
    if con_name and con_name != con_path.split(".")[-1]:
        display(Markdown("   **{}** '{}': '{}'".format(what, con_name, con_path)))
    else:
        display(Markdown("   **{}**: '{}'".format(what, con_path)))

def process_node_service_demands(ndf, what="", separate_table_per_name=False, func=display_df):
    # instead of just displaying dfs below, we could create an edge for each row
    service_demand_idxs = ndf.Parameter.str.lower() == "service demand"
    if any(service_demand_idxs):
        selected_cols = ["Value", "Branch", "Unit"] + year_cols.tolist()
        service_demand_df = ndf.loc[service_demand_idxs]
        service_demand_names = service_demand_df.Value.unique()
        if any(service_demand_names): # named services
            if not separate_table_per_name:
                sd_df = (service_demand_df.sort_values("Value")[selected_cols]
                         .rename(columns={"Value": what if what else "Service Type"}))
                func(sd_df)
            else: # separate table for each demand name
                for name in service_demand_names:
                    sd_df = (service_demand_df.loc[service_demand_df.Value == name, selected_cols]
                             .rename(columns={"Value": what if what else "Service Type"}))
                    
                    func(sd_df)
        else: # components with unnamed paths only
            func(service_demand_df[["Branch"] + year_cols.tolist()].rename(columns={"Parameter":"Component"}))
#             for con_path in service_demand_df["Branch"].values:
#                 process_connection(con_path, None, what="Component")

def create_edges(df, node_path=None, show_df=False):
    print("from node '{}'".format(node_path))
    for index, rdf in df.iterrows():
        target_path = rdf["Branch"]
        print("    connect '{}' ".format(target_path))
        if rdf.index[0] == "Branch":
            target_name = None
        else:
            target_name = rdf.iloc[0]
            if target_path and target_name != target_path.split(".")[-1]:
                print("        via '{}'".format(".".join([node_path, target_name])))
            else:
                target_name = None
        # TODO use node_path, target_path, target_name to create edges
        # TODO how to handle target_path == None?
    if show_df:
        display_df(df)

separate_tables = True # False displays more compactly, but False should be used when creating edges
func = lambda df, node_path: display_df(df)
func = create_edges; separate_tables = True
for nn, ndf in node_dfs.items():
    supply_path = ndf.loc[ndf.Parameter.str.lower() == "service supply", "Branch"]
    if not supply_path.empty:
        node_path = supply_path.values[0]
        display(Markdown("### Node Path: {}".format(node_path)))
        process_node_service_demands(ndf, what="Service Type",
                                     separate_table_per_name=separate_tables,
                                     func=lambda df: func(df, node_path=node_path))
        if nn in tech_dfs:
            process_node_service_demands(pd.concat(tech_dfs[nn].values()), what="Technology",
                                         separate_table_per_name=separate_tables,
                                         func=lambda df: func(df, node_path=node_path))

# TODO insert node and edge construction calls into create_edges function

### Node Path: Canada

from node 'Canada'
    connect 'Canada.Alberta' 


### Node Path: Canada.Alberta

from node 'Canada.Alberta'
    connect 'Canada.Alberta.Residential' 


### Node Path: Canada.Alberta.Residential

...

from node 'Canada.Alberta.Electricity.Generation'
    connect 'Canada.Alberta.Electricity.Generation.Base load' 
from node 'Canada.Alberta.Electricity.Generation'
    connect 'Canada.Alberta.Electricity.Generation.Shoulder load' 
from node 'Canada.Alberta.Electricity.Generation'
    connect 'Canada.Alberta.Electricity.Generation.Peak load' 


### Node Path: Canada.Alberta.Electricity.Generation.Base load

from node 'Canada.Alberta.Electricity.Generation.Base load'
    connect 'Canada.Alberta.Electricity.Generation.Base load.Conventional' 
from node 'Canada.Alberta.Electricity.Generation.Base load'
    connect 'Canada.Alberta.Electricity.Generation.Base load.Conventional' 
        via 'Canada.Alberta.Electricity.Generation.Base load.Renewables'


### Node Path: Canada.Alberta.Electricity.Generation.Base load.Conventional

from node 'Canada.Alberta.Electricity.Generation.Base load.Conventional'
    connect 'Canada.Alberta.Natural Gas' 
        via 'Canada.Alberta.Electricity.Generation.Base load.Conventional.Natural Gas_Single Cycle Gas Turbine'
from node 'Canada.Alberta.Electricity.Generation.Base load.Conventional'
    connect 'Canada.Alberta.Electricity' 
        via 'Canada.Alberta.Electricity.Generation.Base load.Conventional.Solar PV'
from node 'Canada.Alberta.Electricity.Generation.Base load.Conventional'
    connect 'Canada.Alberta.Electricity' 
        via 'Canada.Alberta.Electricity.Generation.Base load.Conventional.Wind'
